In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import pandas as pd
from numpy import genfromtxt
import itertools,random
# np.set_printoptions(threshold=np.nan)
# np.set_printoptions(threshol

Read Embedding File

In [65]:
df=pd.read_csv('../node2vec/emb/gps_1.emb',skiprows=1,sep=' ',header=None)
df=df.sort_values(by=[0],ascending=True)
df=df.rename(columns = {0:'node'})

Read Graph

In [66]:
G=nx.read_edgelist('../datasets/fb_test.txt.1_gps_sample',nodetype=int)
edgeLen=len(G.edges())
edges_G=G.edges()
edges_G=np.asarray(edges_G,dtype=int)
np.random.shuffle(edges_G)

Create Egde Feature vectors for positive class

In [67]:
edge_features_1=np.empty(shape=[0,len(df.columns)-1])
edgeNO=0
for edge in edges_G:
    edgeNO+=1
    node1=df[df['node']==edge[0]].as_matrix().astype(np.float32)[0][1:]
    node2=df[df['node']==edge[1]].as_matrix().astype(np.float32)[0][1:]
    avg=np.mean([node1,node2],axis=0)
    hadamard=np.multiply(node1,node2)
    diff=node1-node2
    wL1=np.absolute(diff)
    WL2=diff**2
    feature=hadamard
    edge_features_1=np.vstack([edge_features_1,hadamard])

In [68]:
G_neg=nx.Graph()
sample_len=len(G.edges())
E=G.edges()
while(len(G_neg.edges())<sample_len):
    n1=random.choice(G.nodes())
    n2=random.choice(G.nodes())
    if(G.has_edge(n1,n2)==False):
        G_neg.add_edge(n1,n2)

In [69]:
edge_features_0=np.empty(shape=[0,len(df.columns)-1])

for edge in G_neg.edges():
    node1=df[df['node']==edge[0]].as_matrix().astype(np.float32)[0][1:]
    node2=df[df['node']==edge[1]].as_matrix().astype(np.float32)[0][1:]
    avg=np.mean([node1,node2],axis=0)
    hadamard=np.multiply(node1,node2)
    diff=node1-node2
    wL1=np.absolute(diff)
    WL2=diff**2
    feature=hadamard
    edge_features_0=np.vstack([edge_features_0,hadamard])

In [70]:
class_0=np.zeros([edgeLen,1])
class_1=np.ones([edgeLen,1])
edge_features_0=np.hstack([edge_features_0,class_0])
edge_features_1=np.hstack([edge_features_1,class_1])
dataset=np.vstack([edge_features_0,edge_features_1])

In [71]:
np.random.shuffle(dataset)
train_X=dataset[:,:dataset.shape[1]-1]
train_Y=dataset[:,dataset.shape[1]-1:]

In [72]:
# G_test=nx.read_adjlist('../datasets/fb_train.txt',nodetype=int)
# i=0
# for x in G_test.nodes():
#     if(G.has_node(x)==True):
#         i+=1
#         if(i%100==0):
#             print i,
# print "I:%d"%(i)

In [73]:
G_test=nx.read_adjlist('../datasets/fb_train.txt',nodetype=int)
test_0=np.empty(shape=[0,len(df.columns)-1])
test_1=np.empty(shape=[0,len(df.columns)-1])
print len(test_0)
sample_len=len(df)
i=0
K=nx.Graph()
while((len(test_0)<sample_len) or (len(test_1)<int(0.5*sample_len))):
    
    i+=1
    n1=random.choice(G_test.nodes())
    n2=random.choice(G_test.nodes())
    if(len(df[df['node']==n1])>0 and len(df[df['node']==n2])>0 and K.has_edge(n1,n2)==False):
        K.add_edge(n1,n2)
        node1=df[df['node']==n1].as_matrix().astype(np.float32)[0][1:]
        node2=df[df['node']==n2].as_matrix().astype(np.float32)[0][1:]
        avg=np.mean([node1,node2],axis=0)
        hadamard=np.multiply(node1,node2)
        diff=node1-node2
        wL1=np.absolute(diff)
        WL2=diff**2
        feature=hadamard
        if((G_test.has_edge(n1,n2)==True) and (len(test_1)<sample_len)):
            test_1=np.vstack([test_1,hadamard])
            if(len(test_1)%10==0):
                print "1:%d"%(len(test_1)),
        elif((G_test.has_edge(n1,n2)==False) and (len(test_0)<sample_len)):
            test_0=np.vstack([test_0,hadamard])
            if(len(test_0)%100==0):
                print "0:%d"%(len(test_0)),
        else:
            pass

print i,len(test_0),len(test_1)

0
0:100 0:200 0:300 0:400 0:500 0:600 1:10 1:20 1:30 1:40 1:50 1:60 1:70 1:80 1:90 1:100 1:110 1:120 1:130 1:140 1:150 1:160 1:170 1:180 1:190 1:200 1:210 1:220

KeyboardInterrupt: 

In [74]:
class_0=np.zeros([test_0.shape[0],1])
class_1=np.ones([test_1.shape[0],1])
test_features_0=np.hstack([test_0,class_0])

test_features_1=np.hstack([test_1,class_1])

testData=np.vstack((test_features_0,test_features_1))
print testData.shape

 (863, 129)


In [75]:
np.random.shuffle(testData)
test_X=testData[:,:testData.shape[1]-1]
test_Y=testData[:,testData.shape[1]-1:]
print test_X.shape
print test_Y.shape
print test_Y

(863, 128)
(863, 1)
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]


In [76]:
X_train=train_X
X_test=test_X
y_train=train_Y
y_test=test_Y

In [77]:
y_test.shape

(863, 1)

In [78]:
from sklearn import svm
clf = svm.SVC(kernel='rbf',C=1.0,probability=False)
clf.fit(X_train,np.ravel(y_train))
y_pred=clf.predict(X_test)
# y_score=clf.predict_proba(X_test)


In [79]:
# from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier(random_state=0)
# clf.fit(X_train,y_train)

# y_pred=clf.predict(X_test)

In [80]:
from sklearn.metrics import f1_score,precision_score,recall_score,roc_auc_score
f1=f1_score(y_test, y_pred, average='macro')

precision=precision_score(y_test, y_pred, average='macro')
recall=recall_score(y_test, y_pred, average='macro')
# auc=roc_auc_score(y_test, y_score)

In [81]:
f1, precision, recall

(0.536515037750803, 0.7496331738437001, 0.55467527386541471)